# Documentation for Clifford-Sim

## Introduction

This code uses the stabilizer formulism to simulate the dynamics of 1-D and 2-D quantum circuits. Specifically, Clifford stabilizers are used, which are made up of Pauli Strings:

* A Pauli String is a tensor product of the 2-D Pauli Spin Matrices and Identity which can act on all qubits of the system
* XZYI is an example of a Pauli String that can be applied to a 4 qubit system
* XZYI can be written as $$\begin{pmatrix}0 & 1 \\ 1 & 0 \end{pmatrix} \otimes \begin{pmatrix}1 & 0 \\ 0 & -1 \end{pmatrix} \otimes \begin{pmatrix}0 & -i \\ i & 0 \end{pmatrix} \otimes \begin{pmatrix}1 & 0 \\ 0 & 1 \end{pmatrix}$$
* The initial tensor product state of four qubits $$\left| \psi \right> = \left|0\right> \otimes \left|0\right> \otimes \left|0\right> \otimes \left|0\right>$$ have a set of many Pauli Strings that preserve the state of the system, which is called the Stabilizer Group
* The Stabilizer Group for this state is generated by the set of Pauli Strings: {ZIII, IZII, IIZI, IIIZ}.



## Using Tableau Objects

The Tableau class creates a Tableau object which stores a binary from of the stabilizer generators into a varible called generator_array. 
$$I \rightarrow 00$$
$$X \rightarrow 1 0$$
$$Y \rightarrow 11$$
$$Z \rightarrow 01$$
There is also a sign_array variable which stores the outcomes of measurements of the system.

In [17]:
# Importing Tableau class and initializing a stabilizer tableau
from tableau import Tableau

# Must give the number of qubits within your system and the starting pure state for the system
# The state given above would initialize to [0,1]
# However, for most of the gates and circuit dynamics in this code, the state should be initialized to Y=[1,1]
# to allow entanglement growth
tab = Tableau(number_of_qubits=4,starting_state=[1,1])

# Printing the object gives the generator array
print(tab)

# Same effect as directly accessing
print(tab.generator_array)

# Need to directly access sign array
print(tab.sign_array)

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]
[[0]
 [0]
 [0]
 [0]]


Each row of the numpy array stores a stabilizer generator in its binary form with a corresponding row in the sign array with a single digit for bit switches depending on a measurement path. Here are some functions added to store and create the Tableau objects:
* save_to_csv(self, output_filename:str, header:str="") 
    - instance method
    - Parameters:   
        * output_filename: file name needed for storage as a .csv file
        * header: optional line of text stored at top of file, if "", the data 
                    starts at line 1 of file
                    
    - The sign array is concatenated to generator array on the right side and stored as one
* initialization_from_csv(cls, filename)
    - class method
    - Parameters: filename: file name that needs to be opened
    - Assumes that sign array is stored with data, and calculates number_of_qubits for the system based on size of leftover array
    - Cannot accept a header line in file

## Using Unitary Gates

To simulate circuit dynamics, specific Unitary Gates are chosen from the Clifford Group. These Unitaries transform Pauli Strings into other Pauli Strings, which allows dynamics to occur while decreasing the amount of data to keep track of. These gates act directly on the object to change the generator array. The gates available in this code are listed:

2 site unitaries:
* controlled_not_left_gate(self, control_position:int)
    - For a 1-D system, control_position should be to the left of target
* controlled_not_right_gate(self, control_position:int)
    - For a 1-D system, control_position should be to the right of target
* nonlocal_controlled_not_gate(self, control_position:int, target_position:int)
    - Used for both 1-D and 2-D to allow non-local interaction between any 2 qubits

1 site unitaries:
* phase_gate(self, position:int)
* hadamard_gate(self, position:int)

Now, let's apply some of these gates to the tableau.

In [18]:
# Apply CNOT Left gate to qubit 0 targeting 1
tab = Tableau(number_of_qubits=4, starting_state=[1,1])
print(tab)
tab.controlled_not_left_gate(control_position=0)
print(tab)

# Changes row 0 from YIII to YXII
# Changes row 1 from IYII to ZYII

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]

[[1 1 1 0 0 0 0 0]
 [0 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]



In [19]:
# Apply CNOT Right gate to qubit 3 targeting 2
tab = Tableau(number_of_qubits=4, starting_state=[1,1])
print(tab)
tab.controlled_not_right_gate(control_position=3)
print(tab)

# Changes row 2 from IIYI to IIYZ
# Changes row 3 from IIIY to IIXY

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1]
 [0 0 0 0 1 0 1 1]]



In [20]:
# Apply Hadamard gate to qubit 2
tab = Tableau(number_of_qubits=4, starting_state=[1,0])
print(tab)
tab.hadamard_gate(position=2)
print(tab)

# Changes row 2 from IIXI to IIZI

[[1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0]]

[[1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]]



Note that there are also methods built in to the class to use random 1 site and 2 site unitaries. These are
* random_cnots_identity(self, position:int)
    - Position is the left most qubit the gate is applied to(ex gate applied to qubits 0 and 1, position=0)
        - Code handles the difference between CNOT Left and Right
    - If these gates are the only ones used for dynamics, the entanglement can only grow if the system is started stabilized by Y
* random_phase_hadamard_identity(self, position:int)
    - Only single sites, so position is just the applied qubit
    - Also 3 gates allowed
* random_nonlocal_cnot_identity(self, position1:int, position2:int)
    - requires the position given for both leftmost qubit and rightmost qubit
    - Similar requirement above for entanglement growth requiring Y initialization

## Measurement

Included is a function that can measure the state of multiple qubits at a time: 

* measure_sites(self, sites:list, measurement:list = [1,0])

The sites argument is a list of the qubit positions the measurement takes place, and the measurement argument applies the binary form of the measurement to all those positions. Each call of the measure sites can only make one type of measurement at a time.

Algorithm:

1. A row array is created the same width as the generator array with all zeros. The measurement is placed within the row array at all positions in the sites list. This creates a Pauli String that represents the measurement.
2. Whether or not this Measurement String commutes with each of the generators in the generator_array is calculated.
3. If they commute, those generators are left alone.
4. If they anticommute, the first anticommuting generator is multiplied (using row_sum shown below) to all the others to make them commute with the measurement.
5. The first anticommuting generator is then replaced with the Measurement String, while a random variable assignes the corresponding value of the sign_array to be 0 or 1 to track measurement outcomes. 0 or 1 denote spin down and up for each measurement type.

Measurements tend to decrease entanglement within the system due to this replacement of the stabiliser generators, which deletes information.

In [21]:
# Applying measurement of Z to qubit 0

# Initializing Tableau to Y
tab = Tableau(number_of_qubits=4, starting_state=[1,1])
print(tab)
print(tab.sign_array)

# Measuring
tab.measure_sites(sites=[0], measurement=[0,1])
print(tab)
print(tab.sign_array)

[[1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]

[[0]
 [0]
 [0]
 [0]]
[[0 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 1]]

[[0]
 [0]
 [0]
 [0]]


You can see how the 0th Pauli String Generator anticommutes with the Measurement Pauli String, so is eventually replaced by it. This updates the sign_array randomly. Rerunning the above cell may change the outcome. Generally in this code, measurements are run one site at a time to be able to measure all sites with some probability of the measurement actually happening. 

These measurements can also be used for a 2-D system if applied uniformly to the whole system since every qubit would have the same probability to be measured. The physical representation of the system would not make this incorrect if this is the case.

## Circuit Dynamics for 1-D

For circuit dynamics, gates need to be applied in layers that denote a time step. For 1-D, there are 2 methods to apply a layer of unitaries
and 1 to apply a layer of random measurement.

### layer_cnots_and_identity(self, is_odd_number_qubits:int, is_applied_odd_bonds:int, periodic_boundary:int)

* Layer of Random CNOT Left, CNOT Right, and Identity
* is_odd_number_qubits: value={0 if even, 1 if odd} 
    * Whether or not number of qubits is odd
* is_applied_odd_bonds: value={0 if left qubit is even index, 1 i left qubit is odd index}
    * For different time steps, the evenness of the time step can alternate interaction bonds to allow all qubits to interact
    * Tip: input is_applied_odd_bonds=time_step mod 2
* periodic_boundary: value{0 if closed boundary, 1 if periodic boundary}
    * Whether or not qubits at endpoints can interact as if in a periodic system


### layer_phase_hadamard_identity(self)

* Layer of Random Phase, Hadamard, and Identity
* One site unitaries make these layers easy to apply
    * method loops over the system size to apply a random gate to each qubit

* This can work for 2-D Systems as well since all gates are one site unitaries

### layer_of_measurement(self, which_pauli:list, measurement_rate = 0.0)

* Layer of one type of measurement with each qubit having a probability between 0 and 1 to be measured
* Loops over all qubit positions, and compares a random variable to the measurement_rate
* This can work for 2-D Systems as well since the measurements are all one site

### Now import 1-D circuit dynamics from dynamics.py

In [22]:
from dynamics import clifford_measure

### clifford_measure(tableau:Tableau, number_time_steps:int, is_odd_number_qubits:int, measurement_rate, which_pauli:list)

Note that this is not an instance method, so the Tableau object is an argument
* number_time_steps: How many layers need to be applied
    * 1 layer is made up of the following:
        * layer_phase_hadamard_identity
        * layer_cnots_identity applied to even bonds
        * layer_of_measurement
        * layer_phase_hadamard_identity
        * layer_cnots_identity applied to odd bonds
        * layer_of_measurement
* is_odd_number_qubits:
    * Needed for layer methods
* measurement_rate:
    * Needed for layer of measurement
* which_pauli:
    * binary form of type of measurement
    * constant throughout all time steps

In [23]:
# Initialize Tableau
tab = Tableau(number_of_qubits=8, starting_state=[1,1])

# Running dynamics
pauli = [1,1]
rate = 0.05
is_odd_number_qubits = tab.number_of_qubits%2
time_steps = 50
clifford_measure(tab, time_steps, is_odd_number_qubits, measurement_rate=rate, which_pauli=pauli)

print(tab)

[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 1]
 [0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1]]



We can see how applying these dynamics tends to fill out the Pauli Strings in the generator array. If the above cell is rerun with higher measurement rates, the generator array is not as able to fill out. This is a visual representation of how unitary dynamics promote entanglement growth in the system, and high measurement resists entanglement growth.

This clifford_measure function can be altered to make your own version of dynamics by applying different gates, or rearranging the layers, or changing the conditions for gates and measurement to be applied. 

## Circuit Dynamics for 2-D

As shown above, the 2-D qubit system requires the application of 2 site unitaries to change since we need to take the shape of the system into account. For a 9 qubit system, the labels are the same for 1-D, but we need to use nonlocal 2 sites to interact adjacent rows of qubits:
$$\begin{matrix}0&1&2\\3&4&5\\6&7&8 \end{matrix} $$
This means we need new functions to build complete layers of 2 site unitaries, but, fortunately, we can still use the 1 site unitaries and 1 site measurement layers.

New functions for 2-D Systems:
* row_layer(self, row:int, is_odd_side_length:int, is_applied_odd_bonds:int)
* column_layer(self, column:int, is_odd_side_length:int, is_applied_odd_bonds:int)
* layer_0_2D(self)
* layer_1_2D(self)
* layer_2_2D(self)
* layer_3_2D(self)

### row_layer(self, row:int, is_odd_side_length:int, is_applied_odd_bonds:int)

This function applies random_nonlocal_cnots_identity to an entire row of the system for every two qubits
* row: index of row in the system
* is_odd_side_length: whether square root of system size is odd
* is_applied_odd_bonds: whether index of first applied qubit is even or odd

In [24]:
# Tableau, 16 qubits, Y
tab = Tableau(number_of_qubits=9, starting_state=[1,1])


# applying layer to row 1 to even bonds
tab.row_layer(row=1, is_odd_side_length=1, is_applied_odd_bonds=0)
print(tab)


# applying layer to row 0 to odd bonds
tab.row_layer(row=0, is_odd_side_length=1, is_applied_odd_bonds=1)
print(tab)


[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]

[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]



For the cell above, when acting on row 0, the unitaries only effect columns 0-2 of the generator array. Each row is a different group of 3.

### column_layer(self, column:int, is_odd_side_length:int, is_applied_odd_bonds:int)

This has the same effect as row_layer, but for columns of qubits in the system
* column: index of column in the system
* is_odd_side_length: whether square root of system size is odd
* is_applied_odd_bonds: whether index of first applied qubit is even or odd

When applying to column 0 for the 9 qubit system, qubits 0,3,6 are able to be interacted.

### layer_?_2D(self)

The 2-D layers of 2 site unitaries are broken up into four types where row and column layers are looped over the entire system:
* Layer_0: Row layers where 0th row is applied even bonds and alternates each row
* Layer_1: Column layers where 0th column is applied odd bonds and alternates each row
* Layer_2: Row layers where 0th row is applied odd bonds and alternates each row
* Layer_3: Column layers where 0th column is applied even bonds and alternates each row


This ensures that the system interacts uniformly as time increases. When using time for loop, time mod 4 can be used to apply each type of layer using match/case or if statements

### two_dimensional_clifford_measure_dynamics(tableau:Tableau, which_case:int, measurement_rate, which_pauli:list)

In [25]:
from dynamics import two_dimensional_clifford_measure_dynamics

Note that this is not an instance method, so the Tableau object is an argument
* which_case: value={0,1,2,3}
    * which type of layer_?_2D to use
* measurement_rate:
    * Needed for layer of measurement
* which_pauli:
    * binary form of type of measurement
* only one time step per function call
    * needs to exist in a loop for long time dynamics

Layers applied:
* layer_phase_hadamard_identity
* layer_?_2D
* layer_of_measurement

In [27]:
# Tableau, 16 qubits, Y
tab = Tableau(number_of_qubits=16,starting_state=[1,1])

# Dynamics
for time in range(100):
    two_dimensional_clifford_measure_dynamics(tableau=tab, which_case=time%4, measurement_rate=0, which_pauli=[1,1])
print(tab)

[[1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1]
 [0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0]
 [1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0]
 [1 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0]
 [1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0]
 [0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 0]
 [0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1]
 [1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 1 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0]
 [0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0]
 [0 1 0 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0]
 [0 1 0 0 1 1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1]
 [1 1 0 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 

## Utilities

This section is dedicated to the manipulation of the generator array by methods in the Tableau class.

### Row Sum

row_sum(self, row_a: int, row_b: int)
* row_a, row_b: indexes of rows in generator array
* uses binary addition to add the two rows together 
* g_function in utilities.py is used to update sign_array
* row_sum algorithm is the same as applying first row's Pauli String as an operator to the second row's Pauli String

Ex. XXIY multiplied to XYZY = $\pm$ IZZI
* [1 0 1 0 0 0 1 1] row_sum with [1 0 1 1 0 1 1 1] = [0 0 0 1 0 1 0 0] = IZZI

In [28]:
# Tableau
tab = Tableau(number_of_qubits=2, starting_state=[1,1])
print(tab)
# row sum row 0 with row 1
tab.row_sum(0,1)
print(tab)

[[1 1 0 0]
 [0 0 1 1]]

[[1 1 0 0]
 [1 1 1 1]]



### Transform to RREF

It is useful to calculate the entanglement entropy and mutual information of the system to transform the generator array of the system to Row Reduced Echelon Form. This is when there is at most 2 leading Pauli Matrices per column in the generator array. The algorithm requires that the two bits representing each part of the Pauli String be manipulated together.

transform_to_rref(self)
* This function does not alter a copy of the generator array; whichever tableau object is input, the actual organization of generator_array is changed
    * Algorithms that use this tend to input a copy of the object to avoid any effects on the actual state, though this should not be required since RREF is equivalent
* uses sort_active_region method and count_paulis function from utilities.py

In [30]:
# Tableau, 8 qubits, Y
tab = Tableau(number_of_qubits=8, starting_state=[1,1])

# Random Unitaries with no measurement
clifford_measure(tableau=tab, number_time_steps=50, is_odd_number_qubits=0, measurement_rate=0, which_pauli=[1,1])
print(tab)

# Transforming to RREF
tab.transform_to_rref()
print(tab)

[[1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1]
 [0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1]
 [1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1]
 [0 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0]
 [1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 0]
 [0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0]
 [0 0 0 1 0 1 1 1 1 0 0 0 1 0 1 0]]

[[1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1]
 [0 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0]
 [0 0 0 1 0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1]
 [0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0]
 [0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1]]



Note: there are occasions where the tableau does not fully look like it is in RREF, but these can usually be fixed by swapping rows or using row_sum on two rows.

## Observables

### Entanglement Entropy

We can divide the system of qubits into multiple subsystems. The reduced density matrices can be pulled out of the generator array to calculate the Von Neumann Entanglement Entropy. For a subsystem A, the entanglement entropy between that A and the complement of A(the rest of the system or $\overline A$) is:
$$S_A=-Tr[\rho _A log(\rho _A)]$$
We calculate this by first rearranging the generator_array columns to be in the form: [$A \overline A$]. Then the array needs to be transformed to RREF. Once in RREF, we can easily calculate the rank of the columns in the array contained within $A$, using the method in the Tableau class. The equation for entanglement entropy is equivalent to:
$$S_A = Rank(A) - Size(A)$$

The get_rank_tableau function counts the number of nonzero rows in the region of qubits that contain $A$. Once the rank is known, the size of $A$ is the number of qubits in $A$.

There are 3 available functions to calculate entanglement entropy. These use the Tableau object, but create a copy of generator_array to not interfere with system state:
* get_entanglement_entropy
* get_entanglement_entropy_for_separated_systems
* get_entanglement_entropy_for_three_separated_systems

get_entanglement_entropy(self, system_end:int, system_start:int = 0)


* Splits system into $A$ and $\overline A$
* system_end: index of last qubit in $A$
* system_start: index of first qubit in $A$

get_entanglement_entropy_for_separated_systems(self, system_A:list, system_B:list)

* Splits system into $A$, $B$, and $\overline {AB}$
* Rearranges into form [$A$ $B$ $\overline {AB}$]
* system_A: length 2 list of start and end indexes of the qubits in $A$
* system_B: length 2 list of start and end indexes of the qubits in $B$

get_entanglement_entropy_for_three_separated_systems(self, system_A:list, system_B:list, system_C:list)

* Splits system into $A$, $B$, $C$, and $\overline {ABC}$
* Rearranges into form [$A$ $B$ $C$ $\overline {ABC}$]
* system_A: length 2 list of start and end indexes of the qubits in $A$
* system_B: length 2 list of start and end indexes of the qubits in $B$
* system_C: length 2 list of start and end indexes of the qubits in $C$

In [36]:
# Tableau, 8 qubits, Y
tab = Tableau(number_of_qubits=8, starting_state=[1,1])

# Entanglement Entropy Initial
# For Bipartition, System_A = [0,3] for System_size(L) = 8
print("Entropy 1",tab.get_entanglement_entropy(system_end=3,system_start=0))

# Random Unitaries with no measurement
clifford_measure(tableau=tab, number_time_steps=50, is_odd_number_qubits=0, measurement_rate=0.00, which_pauli=[1,1])

# Entanglement Entropy After Dynamics
print("Entropy 2", tab.get_entanglement_entropy(system_end=3, system_start=0))

Entropy 1 0
Entropy 2 4


### Mutual Information

The Mutual Information measures the amount of information that is shared solely between multiple systems. There are two functions available, one for two systems, one for three systems:
* get_bipartite_mutual_information(self, system_A:list, system_B:list)
* get_tripartite_mutual_information(self, system_A:list, system_B:list, system_C:list)

The system lists are formatted the same as for the entanglement entropy methods. The algorithm for calculating Mutual Information between A and B is:
$$I_{AB} = S_A + S_B - S_{AB}$$
This can be read as "the information contained in $A$ about $\overline A$ plus the information contained in $B$ about $\overline B$ minus the information contained in $AB$ about $\overline{AB}$". Logically, this leaves only the shared information between the two subsystems.

The Tripartite Mutual Information calculation is a little more complicated, but follows the same logic:
$$I_{ABC}=S_A + S_B + S_C - S_{AB} - S{BC} - S_{AC} + S_{ABC}$$
This measures the shared information between three subsystems.

### Entanglement Negativity

Entanglement Negativity is a measurement of how much the combined density matrix of two subsystems fails to be split into a tensor product of density matrices. The parameters take in the same lists for system A and system B as Entanglement Entropy and Mutual Information. A copy of the object is made inside the Entanglement Negativity Method.

In [41]:
# Tableau, 8 qubits, Y
tab = Tableau(number_of_qubits=16, starting_state=[1,1])

# Mutual Info and EN between A=[0,7] B=[8,15]
print("MI", "EN")
print(tab.get_bipartite_mutual_information([0,7], [8,15]), tab.get_entanglement_negativity([0,7], [8,15]))

# Random Unitaries with no measurement
clifford_measure(tableau=tab, number_time_steps=100, is_odd_number_qubits=0, measurement_rate=0.00, which_pauli=[1,1])

# Values after dynamics
print(tab.get_bipartite_mutual_information([0,7], [8,15]), tab.get_entanglement_negativity([0,7], [8,15]))

MI EN
0 0
14 7


## Example Trial

In example.py, there is an example trial for 2-D measurement. In the start, multiple variables are defined describing how the test is run. For 2-D, there are different ways to divide up the system to measure entanglement entropy, negativity, and mutual information. If $\sqrt{L}$ is divisible by 4, then the system can be evenly divided into horizontal rows. This allows the functions to be used by considering the qubits labelled 1-dimensionally and wrapped into a 2-dimensional plane as described above.

For a 16 qubit system:
* system_A = [0,3]
* system_B = [4,7]
* system_C = [8,11]
* system_D = [12,15]

Additionally, the system can be bipartitioned or divided in half if the system size is even. These are denoted by system_1 and system_2 in the example. There are multiple measurements taken after each time step and added to the data:
* Entanglement Entropy Bipartite: labeled "eeB"
* EE between $AC$ and $\overline{AC}$: labeled "eeAC"
* EE between $BC$ and $\overline{BC}$: labeled "eeBC"
* Mutual Information between $A$ and $C$: labeled "miAC"
* MI between $A$ and $D$: labeled "miAD"
* MI between $B$ and $C$: labeled "miBC"
* Entanglement Negativity between $A$ and $C$: labeled "enAC"
* EN between $A$ and $D$: labeled "enAD"
* EN between $B$ and $C$: labeled "enBC"

The data is arranged in a .npz file with the different data files labeled with the labels above. The first column of each is for each time step of the dynamics, and the rest of the columns are the calculated values for each trials. If there is 100 trials, there will be 101 columns in the data files. These values must be averaged over many trials to show the average behavior of the system in these observables. The number of time steps are chosen so that the system reaches a steady state.